In [6]:
import collections
import numpy as np
import pandas as pd
import math
import random

In [7]:
def run_lru(cache_dict, cache_size, seq, start, box_width, hit_cost, miss_cost):
    # cache_dict=collections.OrderedDict()   
    i = start
    remain_width = box_width
    while remain_width>0:
        if seq[i] in cache_dict.keys():
            remain_width=remain_width-hit_cost
            cache_dict.move_to_end(seq[i], last=False)
        else:
            remain_width=remain_width-miss_cost
            cache_dict[seq[i]]=True
            cache_dict.move_to_end(seq[i], last=False)
            if len(cache_dict.keys())>cache_size:
                cache_dict.popitem(last=True)
        i = i+1
        if i == len(seq):
            break
    return i # Where the box ends/The position of the next request in the sequence 

In [8]:
requests = pd.read_csv('zipf1.08.csv',header=None)
seq=requests[0].tolist()
seq = seq[0:10000]

In [14]:
total_impact=0
rounds=1000
miss_cost=100
boxkinds=7
hist=[0 for _ in range(boxkinds)]
for _ in range(rounds):
    pointer=0
    global_lru=collections.OrderedDict()
    while pointer < len(seq):
        startpointer = pointer
        box=random.randint(0,boxkinds-1)
        hist[box]=hist[box]+1/rounds
        cache_size=2**box
        box_width=miss_cost*cache_size
        # Compartmentalization
        # Load top pages from LRU stack.
        mycache = collections.OrderedDict()
        for pid in global_lru.keys():
            mycache[pid]=True
            mycache.move_to_end(pid, last=True)###########
            if len(mycache)==cache_size:
                break
                
        pointer =run_lru(mycache,cache_size, seq, pointer, box_width, 1, miss_cost)
        endpointer = pointer
            
        # Update global stack
        for x in range(startpointer, endpointer):
            if seq[x] in global_lru.keys():
                global_lru.move_to_end(seq[x], last=False)
            else:
                global_lru[seq[x]]=True
                global_lru.move_to_end(seq[x], last=False)
                
        mi=3*miss_cost*cache_size*cache_size
        total_impact=total_impact+mi
print(total_impact/rounds)
print(hist)

21407293.5
[13.11699999999817, 12.799999999998345, 13.016999999998225, 12.951999999998261, 12.929999999998273, 12.816999999998336, 13.139999999998157]


In [11]:
arr = [(2**i)**2 for i in range(8)]
print(1.5*sum(arr))
print(13*sum(arr[:7]))

32767.5
70993


In [16]:
total_impact=0
boxkinds=7
positions=[]
pointer=0
global_lru=collections.OrderedDict()
box=0
while pointer < len(seq):
    positions.append(pointer)
    startpointer = pointer
    cache_size=2**box
    box_width=miss_cost*cache_size
    # Compartmentalization
    # Load top pages from LRU stack.
    mycache = collections.OrderedDict()
    for pid in global_lru.keys():
        mycache[pid]=True
        mycache.move_to_end(pid, last=True)###########
        if len(mycache)==cache_size:
            break

    pointer =run_lru(mycache,cache_size, seq, pointer, box_width, 1, miss_cost)
    endpointer = pointer

    # Update global stack
    for x in range(startpointer, endpointer):
        if seq[x] in global_lru.keys():
            global_lru.move_to_end(seq[x], last=False)
        else:
            global_lru[seq[x]]=True
            global_lru.move_to_end(seq[x], last=False)

    mi=3*miss_cost*cache_size*cache_size
    total_impact=total_impact+mi
    box=box+1
    if box==boxkinds:
        box=0
print(total_impact)
print(positions)
print(len(positions))

21297900
[0, 1, 4, 8, 21, 53, 184, 806, 807, 809, 815, 839, 894, 1057, 1715, 1716, 1719, 1725, 1748, 1794, 1957, 2482, 2483, 2485, 2490, 2503, 2552, 2671, 3239, 3240, 3242, 3249, 3258, 3305, 3429, 4064, 4065, 4067, 4071, 4082, 4119, 4232, 4892, 4893, 4896, 4901, 4917, 4959, 5111, 5779, 5780, 5782, 5786, 5799, 5842, 5933, 6481, 6482, 6484, 6488, 6500, 6541, 6660, 7247, 7248, 7250, 7254, 7265, 7296, 7404, 8008, 8009, 8011, 8017, 8029, 8070, 8195, 8768, 8769, 8771, 8775, 8794, 8840, 9004, 9726, 9727, 9730, 9735, 9751, 9790, 9916]
91
